In [3]:
import torch
from torch import nn
from torchvision import datasets, transforms

In [4]:
N10K, N50K = 10000, 50000

In [5]:
# N is batch size; D is data dimension;
# H is hidden dimension; C is the number of classes
N, D, H, C = 64, 784, 128, 10

In [6]:
# define a transform to normalize the datae
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [7]:
train_dataset = datasets.MNIST('~/courses/ai/data/', download=True, train=True, transform=transform)
test_dataset = datasets.MNIST('~/courses/ai/data/', download=True, train=False, transform=transform)

In [8]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=N50K, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=N10K, shuffle=True)

In [9]:
x, y = next(iter(train_data_loader))

In [10]:
x.resize_(N50K, D)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [11]:
y_train = torch.zeros([N50K, C])
for i in range(N50K):
    y_train[i][y[i].item()] = 1

In [24]:
y_test = torch.Tensor([N10K, C])
for i in range(N10K):
    y_test[i][y[i].item()] = 1

IndexError: dimension specified as 0 but tensor has no dimensions

In [26]:
# initialize weights and biases
w1 = torch.randn(D, H)
b1 = torch.randn(H)
w2 = torch.randn(H, C)
b2 = torch.randn(H)

In [6]:
learning_rate = 1e-6

In [25]:
for t in range(501):
    # forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t in [100, 200, 300, 400, 500]:
        print(t, loss)

    # backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

RuntimeError: size mismatch, m1: [50000 x 784], m2: [1000 x 100] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:197